# Project One: CRUD Validation Test Script

## Purpose
This notebook provides a structured and repeatable test harness for validating the full **Create, Read, Update, and Delete (CRUD)** functionality of the `AnimalShelter` Python module. The goal of this script is to confirm that all database operations against the `aac.animals` MongoDB collection execute correctly, return expected results, and behave predictably under normal conditions.

This test script is designed to support **professional software engineering practices**, including modular testing, clear verification steps, and consistent output formatting.

---

## Testing Objectives
The objectives of this test script are to:

- Verify successful authentication and connection to MongoDB
- Confirm that new records can be created in the database
- Ensure records can be queried and retrieved accurately
- Validate that existing records can be updated correctly
- Confirm records can be deleted and are no longer retrievable
- Prevent test data from polluting the production dataset by using an identifiable test key

---

## Test Workflow
The script executes the following steps in sequence:

1. **Import the CRUD module** and required dependencies  
2. **Instantiate the `AnimalShelter` class** using authenticated access  
3. **Create** a uniquely identifiable test record in the database  
4. **Read** the record to verify successful insertion  
5. **Update** specific fields in the existing record  
6. **Read** the record again to verify the update  
7. **Delete** the test record from the database  
8. **Perform a final read** to confirm the record has been removed  

Each step includes verification logic to ensure that failures are detected immediately and reported clearly.

---

## Notes
> **Tip:**  
> Initial Create and Read test logic was developed during the Module Four milestone.  
> This notebook expands on that work by incorporating Update and Delete operations, transforming the script into a complete and reusable CRUD validation tool.

---

## Expected Outcome
When executed successfully, this notebook will demonstrate that the `AnimalShelter` CRUD module performs all database operations reliably and in accordance with professional software engineering standards.


In [1]:
# ProjectOneTestScript_Enhanced.ipynb
# Author: Misty Tutkavul
# Course: CS-340 / CS-499 Enhancement
# Purpose:
#   Professionally structured CRUD validation script demonstrating
#   software design and engineering best practices.

from CRUD_Python_Module import AnimalShelter
from pprint import pprint
import sys

# -------------------------------------------------------------
# Configuration
# -------------------------------------------------------------
USERNAME = "aacuser"
PASSWORD = "s3CuR3P@ssw0rd!"
TEST_KEY = "project1-test-record"

TEST_DOCUMENT = {
    "project_test_key": TEST_KEY,
    "name": "Luna",
    "animal_type": "Cat",
    "breed": "Siamese Mix",
    "age_upon_outcome_in_weeks": 45.0,
    "outcome_type": "Adoption",
    "outcome_subtype": "Partner",
    "sex_upon_outcome": "Spayed Female"
}

# -------------------------------------------------------------
# Utility Functions
# -------------------------------------------------------------
def print_section(title: str):
    print("\n" + "-" * 60)
    print(f"{title}")
    print("-" * 60)


def exit_on_failure(message: str):
    print(f"ERROR: {message}")
    sys.exit(1)


# -------------------------------------------------------------
# Connect to MongoDB
# -------------------------------------------------------------
print_section("CONNECT TO MONGODB")

try:
    shelter = AnimalShelter(user=USERNAME, password=PASSWORD)
    print("Connection established successfully.")
except Exception as e:
    exit_on_failure(f"MongoDB connection failed: {e}")


# -------------------------------------------------------------
# CLEANUP (Ensure Idempotent Test Runs)
# -------------------------------------------------------------
shelter.delete({"project_test_key": TEST_KEY})


# -------------------------------------------------------------
# CREATE
# -------------------------------------------------------------
print_section("CREATE OPERATION")

create_result = shelter.create(TEST_DOCUMENT)
print("Create acknowledged:", create_result)

if not create_result:
    exit_on_failure("Create operation failed.")


# -------------------------------------------------------------
# READ (VERIFY CREATE)
# -------------------------------------------------------------
print_section("READ OPERATION (VERIFY CREATE)")

query = {"project_test_key": TEST_KEY}
projection = {"_id": 0, "name": 1, "animal_type": 1, "breed": 1, "outcome_type": 1}

results = shelter.read(query, projection)

if not results:
    exit_on_failure("Read verification failed; no document found.")

print(f"Documents found: {len(results)}")
for doc in results:
    pprint(doc)


# -------------------------------------------------------------
# UPDATE
# -------------------------------------------------------------
print_section("UPDATE OPERATION")

update_data = {
    "name": "Luna Bella",
    "outcome_type": "Transfer"
}

modified_count = shelter.update(query, update_data)
print("Documents modified:", modified_count)

if modified_count != 1:
    exit_on_failure("Update operation did not modify expected document.")


# -------------------------------------------------------------
# READ (VERIFY UPDATE)
# -------------------------------------------------------------
print_section("READ OPERATION (VERIFY UPDATE)")

updated_docs = shelter.read(
    {"project_test_key": TEST_KEY},
    {"_id": 0, "name": 1, "outcome_type": 1}
)

for doc in updated_docs:
    pprint(doc)


# -------------------------------------------------------------
# DELETE
# -------------------------------------------------------------
print_section("DELETE OPERATION")

deleted_count = shelter.delete(query)
print("Documents deleted:", deleted_count)

if deleted_count != 1:
    exit_on_failure("Delete operation failed.")


# -------------------------------------------------------------
# READ (VERIFY DELETE)
# -------------------------------------------------------------
print_section("READ OPERATION (VERIFY DELETE)")

post_delete = shelter.read(query)

print("Post-delete results (should be empty):")
pprint(post_delete)


# -------------------------------------------------------------
# CLOSE CONNECTION
# -------------------------------------------------------------
shelter.close()
print("\nMongoDB connection closed.")
print("=== CRUD TEST COMPLETED SUCCESSFULLY ===")



------------------------------------------------------------
CONNECT TO MONGODB
------------------------------------------------------------
Connection established successfully.

------------------------------------------------------------
CREATE OPERATION
------------------------------------------------------------
Create acknowledged: True

------------------------------------------------------------
READ OPERATION (VERIFY CREATE)
------------------------------------------------------------
Documents found: 1
{'animal_type': 'Cat',
 'breed': 'Siamese Mix',
 'name': 'Luna',
 'outcome_type': 'Adoption'}

------------------------------------------------------------
UPDATE OPERATION
------------------------------------------------------------
Documents modified: 1

------------------------------------------------------------
READ OPERATION (VERIFY UPDATE)
------------------------------------------------------------
{'name': 'Luna Bella', 'outcome_type': 'Transfer'}

--------------------